**WYSA NLP Intern Role Assignment**


Greetings!! My name is Vineet and this is my assignement for the NLP Engineer Intern role at WYSA. Since I am a budding in the field of computational lingusitics and NLP I have taken referred a bunch of sources(classcentral, Medium, Github etc.) to write the code for the proposed problem statement.

I hope you would like my work.

For performing operations on the dataset, I have first tried to create a simplistic label column in the dataset:0---1---2(Negatice-Neutral-Positive).

To do this I have used pandas library to make the apt changes.

In [ ]:
import pandas as pd

path = '/content/dataset-init.csv'
data = pd.read_csv(path)

target_column = 'is_there_an_emotion_directed_at_a_brand_or_product'

data[target_column] = data[target_column].replace({
    'Negative emotion': '0',
    'Positive emotion': '2',
    'No emotion toward brand or product': '1'
})


v1_path = '/content/v1.csv'
data.to_csv(v1_path, index=False)

print("v1 of the dataset with new labels has been saved.")

v1 of the dataset with new labels has been saved.


**Data Pre-Processing & EDA for NLP**

*Data - Cleaning*
- **Tokenization** : involves breaking down text into smaller, manageable units called tokens. These small units act as units for feature extraction.

- **StopWords** : These are recurring words that do not add contextual knowledge to the text, thus are irrelevant and needs to be removed.

- **Casing** : Given the collation of the data, a standardized casing of characters is used. For instance, I have used lowercase.

- **Contractions** : Colloquially contractions(such as aren't,won't) are used due to which negation on the text is often subdued to prevent this it is replaced with standard lexicons.

- **Lemmatization** : Nomalization of disambiguation to base form of language.


In [ ]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')


v1 = '/content/v1.csv'
v2 = '/content/v2.csv'
data = pd.read_csv(v1)


def preprocess_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'[^a-zA-Z\s]', '', text) # Removing special characters and punctuation
        tokens = word_tokenize(text)
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word not in stop_words]
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
        return ' '.join(tokens)
    else:
        return ''


text_columns = data.columns[:2] #Applied to the two columns except the label
data[text_columns] = data[text_columns].applymap(preprocess_text)


data.to_csv(v2, index=False)

print("Preprocessed data saved v2")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
<ipython-input-13-d4ea1ab63dd0>:35: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data[text_columns] = data[text_columns].applymap(preprocess_text)


Preprocessed data saved v2


**Model Training**

Here is how I tackled the main problem. As per the EDA I realised that for every neutral sentiment no product or service was targeted. Thus I trained a model first for sentiment analysis only. Then, if the sentiment is either 0 or 2 then it would be evaluated by another model for determining the the targeted product/service. I split the dataset into training and validation set for the same. I decided on using SVM based on the scatter plots I generated. The accuracy score for the first model is descent but I do not understand the flaw with the second model.(it shows 100% accuracy) ig I made some errors or the model is over-fitting.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, f1_score, recall_score, accuracy_score
import matplotlib.pyplot as plt


v2 = '/content/v2.csv'
data = pd.read_csv(v2)

X = data[data.columns[1]]
y_sentiment = data[data.columns[2]]  # label column

X_train, X_val, y_train, y_val = train_test_split(X, y_sentiment, test_size=0.2, random_state=42)

# Handling NaN values
X_train = X_train.fillna('')
X_val = X_val.fillna('')

# Vectorizing the text data
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_val_tfidf = tfidf.transform(X_val)

# creating classification  model
model_sentiment = SVC(max_iter=1000)
model_sentiment.fit(X_train_tfidf, y_train)

y_pred_sentiment = model_sentiment.predict(X_val_tfidf)


print("Classification Report for Sentiment Analysis:")
print(classification_report(y_val, y_pred_sentiment))
print("F1 Score:", f1_score(y_val, y_pred_sentiment, average='weighted'))
print("Recall Score:", recall_score(y_val, y_pred_sentiment, average='weighted'))
print("Accuracy Score:", accuracy_score(y_val, y_pred_sentiment))


# Creating a second model to predict product/service(0&2)
X_product = X[y_sentiment != 1]  # removing the ones with neutral sentiment
y_product = data[data.columns[1]][y_sentiment != 1]  # product/service labels

X_train_prod, X_val_prod, y_train_prod, y_val_prod = train_test_split(X_product, y_product, test_size=0.2, random_state=42)

# Handling NaN values
X_train_prod = X_train_prod.fillna('')
X_val_prod = X_val_prod.fillna('')
y_train_prod = y_train_prod.fillna('')
y_val_prod = y_val_prod.fillna('')

X_train_prod_tfidf = tfidf.fit_transform(X_train_prod)
X_val_prod_tfidf = tfidf.transform(X_val_prod)

model_product = SVC(max_iter=1000)
model_product.fit(X_train_prod_tfidf, y_train_prod)


y_pred_product = model_product.predict(X_val_prod_tfidf)

print("Classification Report for Product/Service Prediction:")
print(classification_report(y_val_prod, y_pred_product))


Classification Report for Sentiment Analysis:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       103
           1       1.00      0.98      0.99      1059
           2       0.82      1.00      0.90       556

    accuracy                           0.93      1718
   macro avg       0.61      0.66      0.63      1718
weighted avg       0.88      0.93      0.90      1718

F1 Score: 0.901938167828826
Recall Score: 0.9284051222351571
Accuracy Score: 0.9284051222351571
Classification Report for Product/Service Prediction:
                        precision    recall  f1-score   support

                             1.00      1.00      1.00      1039
               android       1.00      1.00      1.00        18
           android app       1.00      1.00      1.00        17
                 apple       1.00      1.00      1.00       134
 apple product service       1.00      1.00      1.00         7
                google       1.00     

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# testing from the test data
input_text = input("Enter a review text: ")
preprocessed_text = preprocess_text(input_text)
input_tfidf = tfidf.transform([preprocessed_text])
sentiment_prediction = model_sentiment.predict(input_tfidf)
print("Predicted Sentiment:", "Positive" if sentiment_prediction[0] == '2' else ("Negative" if sentiment_prediction[0] == '0' else "Neutral"))
if sentiment_prediction[0] in ['0', '2']:
    product_prediction = model_product.predict(input_tfidf)
    print("Targeted Product/Service:", product_prediction[0])

Enter a review text: GSD&amp;M Google #sxsw party with @mention is blowing my mind. So much amazing food, drink, music, &amp; photo booth fun!
Predicted Sentiment: Positive
Targeted Product/Service: google


**Prediction for the Test Data**

In [ ]:
    test = '/content/dataset_test.csv'
    test_data = pd.read_csv(test)
    text_columns = test_data.columns[:0]
    test_data[text_columns] = test_data[text_columns].applymap(preprocess_text)
    X_test_tfidf = tfidf.transform(test_data[test_data.columns[0]])

    sentiment_predictions = model_sentiment.predict(X_test_tfidf)

    product_predictions = []
    for index, sentiment_prediction in enumerate(sentiment_predictions):
        if sentiment_prediction in ['0', '2']:
            product_predictions.append(model_product.predict(X_test_tfidf[index])[0])
        else:
            product_predictions.append("Not Applicable")

    predictions_df = pd.DataFrame({
        'Sentiment': sentiment_predictions,
        'Product/Service': product_predictions
    })

    pred = '/content/test-pred.csv'
    predictions_df.to_csv(pred,index=False)

<ipython-input-28-20675d662794>:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_data[text_columns] = test_data[text_columns].applymap(preprocess_text)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')